<a href="https://colab.research.google.com/github/hamojb/AD_research/blob/main/colab_playaround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from nibabel.testing import data_path
import tensorflow as tf
import nibabel as nib
import os, psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

383107072


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2719928407533008119, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14509932544
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8081766861026454640
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

# Define Helper Functions

In [4]:
def show_slices(slices, col = None):
  """ Function to display row of image slices """
  fig, axes = plt.subplots(1, len(slices))
  for i, slice in enumerate(slices):
      axes[i].imshow(slice.T, cmap=col, origin="lower")
  return

def read_data(data_path: str): 
  """ Function to read in data of each brain scan """
  temp_path = os.path.join(data_path)
  temp_img = nib.load(temp_path)
  temp_data = temp_img.get_fdata()
  return temp_data



def produce_image(data_path: str): 
  """ Function to read in data of each brain scan """
  temp_path = os.path.join(data_path)
  temp_img = nib.load(temp_path)
  return temp_img

# def populate_images(oasis_dir):#, df):
#   """ Function to populate data """ 
#   data = []
#   images = []
#   filenames = []
#   for filename in os.listdir(oasis_dir):
#     print(filename)
#     # if file_str in df
#       # new_data, img = read_data(oasis_dir + filename)
#       # data.append(new_data)
#       # images.append(img)
#   return data, images, filenames

def populate_images(oasis_dir, train_val_test):
  """ Function to populate data """ 
  ret_df = pd.DataFrame()
  count = 0 
  for filename in os.listdir(oasis_dir):
    new_str = filename.split(".")[0].split("_")[0]
    if new_str in train_val_test:
      print(new_str)
      new_data = read_data(oasis_dir + filename)
      ret_df = ret_df.append([new_data,filename.split(".")[0]])
      count += 1
      process = psutil.Process(os.getpid())
      print(process.memory_info().rss)  
      print(count)
    else: 
      continue
    
  return ret_df

def match_data(scan_data, diagnosis_data): 
  empty_df = pd.DataFrame()
  scan_patients = list(set(scan_data.Subject.values))
  for patient in scan_patients:
    temp_scans = scan_data[scan_data['Subject'] == patient]
    temp_scans = temp_scans.sort_values('ID')
    temp_diag = diagnosis_data[diagnosis_data['Subject'] == patient]
    temp_diag = temp_diag.sort_values('ID')
    test = pd.merge_asof(temp_scans, temp_diag, on="ID", allow_exact_matches=True, direction = "forward")
    empty_df = empty_df.append(test)
  empty_df = empty_df.drop(columns = ['Inserted_x', 'Inserted_y', 'Subject_y', 'ageAtEntry'])
  empty_df = empty_df.rename(columns = {"Subject_x":"Subject","FS_FSDATA ID":"MRI_ID"})
  empty_df = empty_df.reset_index().drop(columns = ["index"])
  empty_df = empty_df.dropna()
  return empty_df

Quick demonstation on how to read the data in using the `nibabel` package

In [5]:
# path = os.path.join("/content/drive/MyDrive/oasis_original/OAS30001_Freesurfer53_d0129.mgz.nii.gz")
# img = nib.load(path)
# data = img.get_fdata()

In [6]:
# slice_0 = data[128, :, :]
# slice_1 = data[:, 128, :]
# slice_2 = data[:, :, 128]
# show_slices([slice_0, slice_1, slice_2], "gray")

Define functions to read all data in from `oasis_originals`. 

# Read data and store in variable. 

In [7]:
clinical_data = pd.read_csv("/content/drive/MyDrive/extra_data/clinical_data.csv")
clinical_data['ID'] = clinical_data['ADRC_ADRCCLINICALDATA ID'].str.split('_ClinicalData_', 1, expand=True)[1]
clinical_data = clinical_data.drop(columns = ['ADRC_ADRCCLINICALDATA ID', 'Date', 'Subject.1', 'id'])
clinical_data['Inserted'] = pd.to_datetime(clinical_data['Inserted'])
clinical_data['ID'] = clinical_data['ID'].str.extract('(\d+)', expand=False).astype(int)

In [8]:
scan_info = pd.read_csv("/content/drive/MyDrive/extra_data/freesurfers_mri_info.csv")
scan_info['ID'] = scan_info['FS_FSDATA ID'].str.split('_Freesurfer', 1, expand = True)[1]
scan_info = scan_info.drop(columns = ['Session', 'FS Date'])
scan_info.ID = scan_info.ID.str[3:]
scan_info['Inserted'] = pd.to_datetime(scan_info['Inserted'])
scan_info['ID'] = scan_info['ID'].str.extract('(\d+)', expand=False).astype(int)

In [9]:
scan_and_diag = match_data(scan_info, clinical_data)
scans_sorted = scan_and_diag.sort_values(["Subject", "ID"]).reset_index().drop(columns = ["index", "ID"])
scans_sorted

,MRI_ID,Subject,cdr
0,OAS30001_Freesurfer53_d0129,OAS30001,0.0
1,OAS30001_Freesurfer53_d0757,OAS30001,0.0
2,OAS30001_Freesurfer53_d2430,OAS30001,0.0
3,OAS30001_Freesurfer53_d3132,OAS30001,0.0
4,OAS30002_Freesurfer53_d0653,OAS30002,0.0
...,...,...,...
1824,OAS31168_Freesurfer53_d2526,OAS31168,0.0
1825,OAS31169_Freesurfer53_d0620,OAS31169,0.5
1826,OAS31170_Freesurfer53_d2410,OAS31170,1.0
1827,OAS31172_Freesurfer53_d0407,OAS31172,0.0


In [10]:
from collections import Counter
Counter(scan_and_diag['cdr'])

Counter({0.0: 1460, 0.5: 267, 1.0: 82, 2.0: 17, 3.0: 3})

We will now check to make sure that in our directory of images we have the correct MRI_IDs to link to the existing data. 

In [11]:
num = 0
for i in os.listdir(r'/content/drive/MyDrive/oasis_original/'):
  if i.split(".")[0] in scans_sorted.MRI_ID.values: 
    num += 1
print(num)

1824


In [12]:
subj_ids = list(set(scans_sorted["Subject"]))
inv_test = int(0.15*len(subj_ids))
print(inv_test)

140


In [13]:
from random import shuffle
shuffle(subj_ids)
train_val = subj_ids[inv_test:]
train = train_val[inv_test:]
val = train_val[:inv_test]
test = subj_ids[:inv_test]

In [14]:
# len(train)/8

In [15]:
len(val)

140

In [16]:
len(test)

140

In [17]:
overlaps = set(train).intersection(val, test)
if len(overlaps) == 0: 
  print("CLEAR: No overlaps")
else: 
  print("ERROR: Overlaps exist")
  print(overlaps)

CLEAR: No overlaps


In [18]:
len(val)/2

70.0

In [ ]:
scans_sorted[scans_sorted.Subject in train]

In [ ]:
scans_sorted[scans_sorted.Subject in val]

In [ ]:
scans_sorted[scans_sorted.Subject in test]

Good enough... for now 

In [19]:
val1 = val[:int(len(val)/2)]
val2 = val[int(len(val)/2):]

In [20]:
val1

['OAS30316',
 'OAS30270',
 'OAS30491',
 'OAS30184',
 'OAS30484',
 'OAS30927',
 'OAS30739',
 'OAS30218',
 'OAS30652',
 'OAS30728',
 'OAS30688',
 'OAS30200',
 'OAS30880',
 'OAS30027',
 'OAS31007',
 'OAS30083',
 'OAS30966',
 'OAS30724',
 'OAS30667',
 'OAS30790',
 'OAS30278',
 'OAS30009',
 'OAS30293',
 'OAS30943',
 'OAS30694',
 'OAS30947',
 'OAS30268',
 'OAS30474',
 'OAS30692',
 'OAS30356',
 'OAS31003',
 'OAS30754',
 'OAS30088',
 'OAS30456',
 'OAS30410',
 'OAS30151',
 'OAS31000',
 'OAS30636',
 'OAS30736',
 'OAS30878',
 'OAS30170',
 'OAS30925',
 'OAS30524',
 'OAS30451',
 'OAS30664',
 'OAS30900',
 'OAS30352',
 'OAS30212',
 'OAS30500',
 'OAS31090',
 'OAS30702',
 'OAS30153',
 'OAS30084',
 'OAS30473',
 'OAS30375',
 'OAS30394',
 'OAS31022',
 'OAS30929',
 'OAS30361',
 'OAS30392',
 'OAS30074',
 'OAS30093',
 'OAS31028',
 'OAS30432',
 'OAS30141',
 'OAS30226',
 'OAS30419',
 'OAS30367',
 'OAS31165',
 'OAS30259']

In [ ]:
# directory = r'/content/drive/MyDrive/oasis_original/'
# val_data1 = populate_images(directory, val1)
# # val_data2 = populate_images(directory, val2)

# test_data = populate_images(directory, test)
# train_data = populate_images(directory, train)


OAS30074
1115893760
1
OAS30074


/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


1250197504
2
OAS30074
1385590784
3
OAS30083
1519902720
4
OAS30083
1654755328
5
OAS30083
1789067264
6
OAS30084
1923379200
7
OAS30084
2057691136
8
OAS30088
2192273408
9
OAS30093
2326855680
10
OAS30093
2461167616
11
OAS30093
2595479552
12
OAS30212
2729791488
13
OAS30212
2864103424
14
OAS30212
2998685696
15
OAS30218
3132997632
16
OAS30218
3267309568
17
OAS30226
3401621504
18
OAS30293
3536203776
19
OAS30293
3670515712
20
OAS30410
3804827648
21
OAS30451
3939139584
22
OAS30456
4073451520
23
OAS30473
4207763456
24
OAS30473
4342075392
25
OAS30474
4476657664
26
OAS30652
4610969600
27
OAS30652
4745281536
28
OAS30692
4879593472
29
OAS30694
5013905408
30
OAS30694
5148217344
31
OAS30880
5282529280
32
OAS30880
5416841216
33
OAS31090
5551423488
34
OAS31090
5685735424
35
OAS31090
5820047360
36
OAS31090
5954359296
37
OAS31165
6088941568
38
OAS31165
6223253504
39
OAS30009
6357565440
40
OAS30009
6491877376
41
OAS30009
6626189312
42
OAS30027
6760501248
43
OAS30027
6895083520
44
OAS30027
7029395456
45
OAS30

In [ ]:
train_data